# Import and process

In [ ]:
%load_ext autoreload
%autoreload 2
import logic.processing as lp
import string

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [2]:
data = lp.load_data('Combined Data.csv')

In [3]:
df = lp.preproc(data, bi = True)

In [4]:
df.sample(20)

,Unnamed: 0,statement,status,clean,label
27785,27785,Grad school isn’t really the easiest place to ...,Stress,grad school ’ really easiest place parent alon...,0
2924,2924,Got a gift from fujifilm grgr video kmrn fypðŸ˜‚,Normal,get gift fujifilm grgr video kmrn fypðÿ˜‚,1
5447,5447,"dream of being a kindergarten teacher, where a...",Normal,dream kindergarten teacher tell teach science ...,1
942,942,I should have wake up early today bro. Tomorro...,Normal,wake early today bro tomorrow play day,1
49843,49843,"Dreadddd When the caffeine cravings, sleepless...",Bipolar,dreadddd caffeine craving sleeplessness though...,0
45212,45212,can t sleep how frustrating,Normal,sleep frustrate,1
9550,9550,"...if you do not have a goal, life is a chore....",Depression,goal life choremy goal kill life suck courage ...,0
49764,49764,I’m new I’m very recently diagnosed (in conjun...,Bipolar,’ new ’ recently diagnose conjunction w asd ’ ...,0
51793,51793,So tired of everything I am so goddamn tired ...,Personality disorder,tire everything goddamn tire everything look f...,0
12847,12847,I do not have a single person that actually ca...,Suicidal,single person actually care way accept fact lo...,0


# Model : Naive Baysian

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve, precision_score, accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [6]:
# pipe1 = make_pipeline(TfidfVectorizer(ngram_range=(1,1)),
#                      MultinomialNB(alpha=0.1))

In [7]:
pipe2 = make_pipeline(TfidfVectorizer(ngram_range=(1,1), max_df= 0.9),
                     MultinomialNB(alpha=0.01))

In [8]:
X = df['clean']
y = df['label']

In [10]:
X.isna().sum()

0

In [ ]:
cv1 = cross_validate(pipe2, X,y, scoring = ['accuracy','precision','recall'])

In [ ]:
cv1

In [ ]:
cvp = cross_val_predict(pipe2,X,y,method='predict_proba')

In [ ]:
cvp.T[1]

In [ ]:
proba_norm = cvp.T[1]

In [ ]:
pre, rec, thresh = precision_recall_curve(y, proba_norm)

In [ ]:
scores = pd.DataFrame({'precision' : pre[:-1],
                      'recall' : rec[:-1],
                      'threshold' : thresh})

In [ ]:
scores

In [ ]:
plt.plot(scores['recall'], scores['precision']);

In [ ]:
seuil = scores[scores['recall'] >= 0.85].threshold.max()
seuil

In [ ]:
def custom(proba,threshold):
    return (proba > threshold).astype(int)

In [ ]:
updated = custom(proba_norm, seuil)

In [ ]:
updated

In [ ]:
recall_new = recall_score(y, updated)
recall_new

In [ ]:
precision_new = precision_score(y, updated)
precision_new

In [ ]:
accuracy_score(y,updated)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y, updated)

## Grid searching

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
TfidfVectorizer()

In [ ]:
pipe1.get_params()

In [ ]:
TfidfVectorizer()

In [ ]:
params = {'tfidfvectorizer__ngram_range' : ((1,1),(1,2),(2,2)),
          'tfidfvectorizer__max_df' : [0.9, 0.95],
          'multinomialnb__alpha' : [0.01, 0.1, 1]
         }


In [ ]:
grid = GridSearchCV(pipe2,
                    params,
                    scoring = 'recall',
                    n_jobs=-1,
                    verbose=2
                   )

In [ ]:
grid.fit(X,y)

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_